___
# Atividade: <p> <center> Titanic - Classificador com Teorema de Bayes
___

## Aula 04

**Objetivo da aula:** Ao final desta aula, o aluno deve ser capaz de:
 * Entender conceitos básicos do Teorema de Bayes; e
 * Compreender e implementar um Classificador Naive Bayes.

___
<div id="indice"></div>

## Índice

- [Titanic](#titanic) <p>
    
- [Abrindo a base de dados](#dados)
    - [Mostrando aos **Pandas** quais variáveis do *dataframe* devem ser tratadas como categóricas](#categoria) <p>
    
- [Objetivo](#obj) <p>
    
- [Classificador sem considerar qualquer característica do passageiro](#apriori)    <p>
    
- [Classificador Naive-Bayes: predizer se seria sobrevivente considerando o perfil do passageiro](#naivebayes)
    - [Teorema de Bayes](#teo)<p>
    
- [Exercícios](#exercicios)
    - [Exercício 1](#ex1)
    - [Exercício 2](#ex2)
    - [Exercício 3(i)](#ex3i)
    - [Exercício 3(ii)](#ex3ii)
    - [Exercício 4](#ex4) <p>
    
- [Classificando os passageiros](#class)
    - [Exercício 5](#ex5) <p>
    
- [Análise de sentimento](#sentimento)
    - [Exercício 6](#ex6)
    - [Exercício 7](#ex7) <p>
    
- [Para casa: Estudo Complementar](#casa)
    - [Classificador Naive-Bayes com a variável `Pclass`](#classPclass)
    - [Separando conjunto de dados em Treinamento (train)  e Teste (test)](#testtrain)
___

___
<div id="titanic"></div>


## Titanic

<img src="titanic.jpeg">

Fonte: extraído da internet

Entre as muitas competições disponíveis na plataforma Kaggle, encontra-se a de *Machine Learning* do Titanic conforme apresentado neste [link](https://www.kaggle.com/c/titanic/data).

A base de dados apresentada no arquivo `titanic.xlsx` contém informação de 891 passageiros para os quais foram mensurados as seguintes informações (as categorias serão mantidas em inglês assim como os nomes das variáveis):
 
 * `Survived`: indica se passageiro sobreviveu ou não.
    - 0 = No,
    - 1 = Yes.
 * `Pclass`: indica a classe da passagem (*ticket*).
    - 1 = 1st, 
    - 2 = 2nd, 
    - 3 = 3rd.
 * `Sex`: sexo do passageiro. 
    - female,
    - male.
 * `Age`: idade em anos.
 * `Sibsp`: número de irmãos e/ou cônjuges a bordo do Titanic.
 * `Parch`: número de pais e/ou filhos a bordo do Titanic.
 * `Ticket`: número do *ticket*.
 * `Fare`: tarifa do passageiro.
 * `Cabin`: número da cabine.
 * `Embarked`: portão de embarque.
    - C = Cherbourg, 
    - Q = Queenstown, 
    - S = Southampton.


**Notas:**
A variável `Pclass` pode ser considerada uma *proxy* da situação socioeconômico do passageiro, ou seja, 
1st = Upper, 2nd = Middle,  3rd = Lower.

[Volta ao Índice](#indice)

___
<div id="dados"></div>

## Abrindo a base de dados

Carregando algumas bibliotecas:

In [599]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [600]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\Users\PH\Desktop\Repositorios\cdados


Carregando os dados do arquivo `titanic.xlsx`:

In [601]:
dados = pd.read_excel('titanic.xlsx')

Para saber quantas linhas (passageiros) e colunas (variáveis) há no *DataFrame* nomeado `dados`:

In [602]:
dados.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [603]:
dados.Name.count()

891

<div id="categoria"></div>

### Mostrando aos Pandas quais variáveis do *dataframe* devem ser tratadas como categóricas

O pandas não tem como saber quais variáveis são qualitativas ao carregar os dados. <br>
Então precisamos modificar as colunas explicitamente.

Já deixamos isso pronto para você nesta avaliação:

In [604]:
dados['Survived'] = dados['Survived'].astype('category')
dados['Pclass'] = dados['Pclass'].astype('category')
dados['Sex'] = dados['Sex'].astype('category')
dados['Cabin'] = dados['Cabin'].astype('category')
dados['Embarked'] = dados['Embarked'].astype('category')

Agora vamos renomear os rótulos das variáveis qualitativas `Survived` e `Pclass` com comando `.cat.categories`:

In [605]:
dados.Sex.cat.categories = ['female','male']
dados.Pclass.cat.categories = ['1st','2nd','3rd']
dados.Survived.cat.categories = ['No', 'Yes']

C:\Users\PH\AppData\Local\Temp\ipykernel_744\617429953.py:1: FutureWarning: Setting categories in-place is deprecated and will raise in a future version. Use rename_categories instead.
  dados.Sex.cat.categories = ['female','male']
C:\Users\PH\AppData\Local\Temp\ipykernel_744\617429953.py:2: FutureWarning: Setting categories in-place is deprecated and will raise in a future version. Use rename_categories instead.
  dados.Pclass.cat.categories = ['1st','2nd','3rd']
C:\Users\PH\AppData\Local\Temp\ipykernel_744\617429953.py:3: FutureWarning: Setting categories in-place is deprecated and will raise in a future version. Use rename_categories instead.
  dados.Survived.cat.categories = ['No', 'Yes']


In [606]:
dados.Sex = pd.Categorical(dados.Sex, categories=['female','male'], ordered=False)
dados.Pclass = pd.Categorical(dados.Pclass, categories=['1st','2nd','3rd'], ordered=True)
dados.Survived = pd.Categorical(dados.Survived, categories=['No', 'Yes'], ordered=False)

O *dataFrame* `dados` que deve considerar nos passos a seguir:

In [607]:
dados.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,No,3rd,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,Yes,1st,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,Yes,3rd,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


[Volta ao Índice](#indice)

___
<div id="obj"></div>

## Objetivo

Imagine que você com as suas características estivesse no Titanic no dia no naufrágio. <br>
Será que você teria sido um sobrevivente ou não??

Assim, o objetivo dessa atividade é classificar um "novo" passageiro do Titanic como **Sobrevivente** ou **não Sobrevivente** levando em considerando (ou seja, aprendendo) o perfil dos sobreviventes e não sobreviventes que de fato estavam no Titanic.

[Volta ao Índice](#indice)

<div id="apriori"></div>

___
## Classificador sem considerar qualquer característica do passageiro

Como você poderia dar um "chute" inicial para avaliar se um novo passageiro seria ou não sobrevivente no Titanic?

Podemos usar uma probabilidade conhecida como **a priori** que não leva em consideração qualquer característica do passageiro. Essa probabilidade é dada por:

$$\qquad P(sobrevivente)\qquad \mbox{e} \qquad P(não\hspace{1mm}sobrevivente).$$

Assuma $S$ como evento ser sobrevivente e $S^c$ como evento não ser sobrevivente.

Agora, calcule essas probabilidades a partir da base de dados. Lembra que você aqui pode usar o comando `value_counts()`.

In [608]:
# Tabela de frequências relativas => Probabilidades marginais
dados.Survived.value_counts(True, sort=False).round(4)

No     0.6162
Yes    0.3838
Name: Survived, dtype: float64

E aí? Se chegasse alguém agora para ser um passageiro do Titanic, esse seria classificado como sobrevivente ou não sobrevivente?

E se chegassem 1.000 passageiros?

In [609]:
# COLOQUE SUA RESPOSTA AQUI
# Seria classificado como não sobrevivente
# 616 seriam classificados como não sobreviventes e 384 como sobreviventes

[Volta ao Índice](#indice)

___
<div id="naivebayes"></div>

## Classificador Naive-Bayes: predizer se seria sobrevivente a partir do seu perfil

Um **Classificador Naive-Bayes** desempenha a função de realizar classificações, neste caso, em categorias como **sobrevivente** ou **não sobrevivente**, com base em características específicas dos passageiros. <br>
É fundamental considerar variáveis do passageiro, também conhecidas como *features*, que apresentem alguma associação com a variável *target* `Survived`, que é responsável pela categorização. <br>
Vale destacar a importância de escolher *features* do passageiro que estejam relacionadas (dependentes) com a variável *target*, pois isso possibilita uma classificação mais precisa.

<div id="teo"></div>

### Teorema de Bayes

Vamos entender o **Teorema de Bayes** utilizando a característica do passageiro descrita na variável `Sex`. <br>
Ainda, vamos utilizar os eventos:
 * $S$ para sobrevivente, 
 * $S^c$ para não sobrevivente, 
 * $F$ para feminino e 
 * $M$ para masculino.

No caso, a probabilidade condicional de **ser sobrevivente** dado cada categoria da variável `Sex` pode ser escrita como:

$$P(S|F)=\frac{P(S \cap F)}{P(F)}=\frac{P(F|S)P(S)}{P(F)}\hspace{3cm} (I)$$

e

$$P(S|M)=\frac{P(S \cap M)}{P(M)}=\frac{P(M|S)P(S)}{P(M)}\hspace{2.5cm} (II)$$


Por consequência, a probabilidade de **não ser sobrevivente** dado cada categoria da variável `Sex` pode ser escrita como:

$$P(S^c|F)=1-P(S|F)$$

e

$$P(S^c|M)=1-P(S|M)$$


Uma vez que essas probabilidades descritas em $(I)$ e $(II)$, ou seja, $P(S|F)$ e $P(S|M)$, tenham sido determinadas por meio de cálculos, elas se tornam ferramentas valiosas para a classificação de um novo passageiro como **sobrevivente** ou **não**, com base em suas características pessoais (aqui expressas apenas pela *feature* `Sex`).

Agora, vamos obter todas as probabilidade envolvidas do lado direito das equações $(I)$ e $(II)$ acima descritas utilizando os resultados amostrais.

[Volta ao Índice](#indice)

___
<div id="exercicios"></div>

## EXERCÍCIOS

Vejam as probabilidades que devem calcular a partir da base de dados para que possa obter as equações $(I)$ e $(II)$ conforme descritas acima. <br>
Para facilitar, vamos obter tais probabilidades diluídas em quatro exercícios. <br>
Vamos lá! **Passo-a-passo**:

<div id="ex1"></div>

### Exercício 1:

Calcule as probabilidades $P(S)$, $P(S^c)$, $P(F)$ e $P(M)$.  <br>
Aqui você pode usar o comando `value_counts()`, lembra? <br>
OBS.: Guarde essas probabilidades nas respectivas variáveis do python `P_S`, `P_Sc`, `P_F` e `P_M`.

In [610]:
# Item resolvido a seguir. Use como referência para demais itens!

In [611]:
tabela_survived = dados.Survived.value_counts(True, sort=False)
P_Sc = tabela_survived[0]
P_S = tabela_survived[1]

print(f'Probabilidade a priori de não sobreviver: {P_Sc:0.5g}')
print(f'Probabilidade a priori de sobreviver: {P_S:0.5g}')

Probabilidade a priori de não sobreviver: 0.61616
Probabilidade a priori de sobreviver: 0.38384


In [612]:
dados.Sex.value_counts(True, sort=False)

female    0.352413
male      0.647587
Name: Sex, dtype: float64

In [613]:
prob_sex = dados.Sex.value_counts(True, sort=False)
P_F = prob_sex[0]
P_M = prob_sex[1]

print(f'Probabilidade de ser do sexo feminino: {P_F:0.5g}')
print(f'Probabilidade de ser do sexo masculino: {P_M:0.5g}')

Probabilidade de ser do sexo feminino: 0.35241
Probabilidade de ser do sexo masculino: 0.64759


[Volta ao Índice](#indice)

 <div id="ex2"></div>

### Exercício 2:

Construa um conjunto de dados apenas com os sobreviventes e um outro apenas com os não sobreviventes. <br>
OBS.: Guarde esses conjuntos de dados nas seguintes variáveis do python `dados_S` e `dados_Sc`, respectivamente.

In [614]:
# COLOQUE SEU CODIGO AQUI
dados_S = dados.loc[dados.Survived == 'Yes' , :]
dados_Sc = dados.loc[dados.Survived == 'No' , :]

In [615]:
dados_S

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,2,Yes,1st,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,Yes,3rd,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,Yes,1st,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
8,9,Yes,3rd,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,Yes,2nd,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C
...,...,...,...,...,...,...,...,...,...,...,...,...
875,876,Yes,3rd,"Najib, Miss. Adele Kiamie ""Jane""",female,15.0,0,0,2667,7.2250,NaN,C
879,880,Yes,1st,"Potter, Mrs. Thomas Jr (Lily Alexenia Wilson)",female,56.0,0,1,11767,83.1583,C50,C
880,881,Yes,2nd,"Shelley, Mrs. William (Imanita Parrish Hall)",female,25.0,0,1,230433,26.0000,NaN,S
887,888,Yes,1st,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S


In [616]:
dados_Sc

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,No,3rd,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
4,5,No,3rd,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,No,3rd,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,No,1st,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,No,3rd,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
884,885,No,3rd,"Sutehall, Mr. Henry Jr",male,25.0,0,0,SOTON/OQ 392076,7.0500,NaN,S
885,886,No,3rd,"Rice, Mrs. William (Margaret Norton)",female,39.0,0,5,382652,29.1250,NaN,Q
886,887,No,2nd,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
888,889,No,3rd,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S


[Volta ao Índice](#indice)

 <div id="ex3i"></div>

### Exercício 3(i):
Entre os sobreviventes: calcule $P(F|S)$ e $P(M|S)$ a partir da base de dado `dados_S`. <br>
Aqui pode usar o comando `value_counts()`. <br>
OBS.: Guarde essas probabilidades nas respectivas variáveis do python `P_F_dado_S` e `P_M_dados_S`.
 
<p> 
 
*Respostas esperadas:* <br> 
*Probabilidade de ser do sexo feminino entre os sobreviventes: 0.68129*  <br> 
*Probabilidade de ser do sexo masculino entre os sobreviventes: 0.31871*

In [617]:
# COLOQUE SEU CODIGO AQUI
P_XXX_dado_S = dados_S.Sex.value_counts(True, sort=False)
P_F_dado_S = P_XXX_dado_S[0]
P_M_dado_S = P_XXX_dado_S[1]

In [618]:
P_F_dado_S

0.6812865497076024

In [619]:
P_M_dado_S

0.31871345029239767

[Volta ao Índice](#indice)

<div id="ex3ii"></div>

### Exercício 3(ii):

Entre os não sobreviventes: calcule $P(F|S^c)$ e $P(M|S^c)$ a partir da base de dados `dados_Sc`.  <br>
Aqui pode usar o comando `value_counts()`.  <br>
OBS.: Guarde essas probabilidades nas respectivas variáveis do python `P_F_dado_Sc` e `P_M_dados_Sc`.
  
<p> 
 
*Respostas esperadas:* <br>
*Probabilidade de ser do sexo feminino entre os não sobreviventes: 0.14754* <br>
*Probabilidade de ser do sexo masculino entre os não sobreviventes: 0.85246*

In [620]:
# COLOQUE SEU CODIGO AQUI
P_XXX_dado_Sc = dados_Sc.Sex.value_counts(True, sort=False)
P_F_dado_Sc = P_XXX_dado_Sc[0]
P_M_dado_Sc = P_XXX_dado_Sc[1]

In [621]:
P_F_dado_Sc

0.14754098360655737

In [622]:
P_M_dado_Sc

0.8524590163934426

[Volta ao Índice](#indice)

<div id="ex4"></div>

### Exercício 4:

Com todas essas probabilidades, calcule $P(S|F)$, $P(S^c|F)$, $P(S|M)$ e $P(S^c|M)$ seguindo as equações (I) e (II). <br>
OBS.: Guarde essas probabilidades nas respectivas variáveis do python `P_S_dado_F`, `P_Sc_dado_F`, `P_S_dado_F` e `P_Sc_dado_M`.
 
<p> 
 
*Respostas esperadas:* <br> 
*Probabilidade de ser sobreviventdo se feminino: 0.74204* <br>
*Probabilidade de ser não sobrevivente se feminino: 0.25796* <br>
*Probabilidade de ser sobreviventdo se masculino: 0.18891* <br>
*Probabilidade de ser não sobrevivente se masculino: 0.81109*

In [623]:
# COLOQUE SEU CODIGO AQUI
P_S_dado_F = P_F_dado_S*P_S/P_F
P_Sc_dado_F = 1 - P_S_dado_F
P_S_dado_M = P_M_dado_S*P_S/P_M
P_Sc_dado_M = 1 - P_S_dado_M

In [624]:
P_S_dado_F

0.7420382165605095

In [625]:
P_Sc_dado_F

0.2579617834394905

In [626]:
P_S_dado_M

0.18890814558058924

In [627]:
P_Sc_dado_M

0.8110918544194108

[Volta ao Índice](#indice)

<div id="class"></div>

___
## Classificando os passageiros 

<div id="ex5"></div>

### Exercício 5:

Considerando seu **Classificador Naive-Bayes**, classifique cada passageiro do *dataFrame* `dados`como `No` ou `Yes` considerando apenas a variável `Sex`.

Conforme apresentado na Seção Classification descrita neste [link](https://sebastianraschka.com/Articles/2014_naive_bayes_1.html), a classificação considerando a variável `Sex` dará da seguinte forma:

**Classificação se passageiro é Feminino:** 
$$\begin{array}{lll} 
    \mbox{Se } P(S|F)>P(S^c|F) & \mbox{então } & \mbox{Classificar como Sobrevivente} \\
                                               & \mbox{caso contrário } & \mbox{Classificar como Não Sobrevivente} 
   \end{array}$$

<br>

**Classificação se passageiro é Masculino:** 
$$\begin{array}{lll} 
    \mbox{Se } P(S|M)>P(S^c|M) & \mbox{então } & \mbox{Classificar como Sobrevivente} \\
                                               & \mbox{caso contrário } & \mbox{Classificar como Não Sobrevivente} 
   \end{array}$$

No *dataframe* `dados`, **faça** a classificação de cada passageiro de acordo com as restrições acima e **aguarde-a** em uma nova coluna chamada `Boot`. 

In [628]:
# COLOQUE SEU CODIGO AQUI
dados['BootSex'] = np.where(dados.Sex == 'female', 'Yes', 'No')

In [629]:
dados

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,BootSex
0,1,No,3rd,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,No
1,2,Yes,1st,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Yes
2,3,Yes,3rd,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Yes
3,4,Yes,1st,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Yes
4,5,No,3rd,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,No,2nd,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,No
887,888,Yes,1st,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,Yes
888,889,No,3rd,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S,Yes
889,890,Yes,1st,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,No


[Volta ao Índice](#indice)

<div id="sentimento"></div>

___
## Análise de sentimento

<div id="ex6"></div>

### Exercício 6:

Faça uma análise de sentimento cruzando a classificação do robô (armazenada em `Boot`) com a real classificação do passageiro (armazenada em `Survived`) utilizando a função `pd.crosstab()`.

Calcule as probabilidades:
 * de acerto;
 * falso positivo;
 * falso negativo.
 
Analise os resultados.

<br>

*Resposta esperada:*<br>
Probabilidade de acerto: 78.676% <br>
Probabilidade de falso positivo: 14.754% <br>
Probabilidade de falso negativo: 31.871%

In [630]:
# COLOQUE SEU CODIGO AQUI
pd.crosstab(dados.BootSex, dados.Survived)

Survived,No,Yes
BootSex,,
No,468,109
Yes,81,233


In [631]:
print((468+233)/(468+233+81+109))
print((81)/(468+81))
print((109)/(109+233))

0.7867564534231201
0.14754098360655737
0.31871345029239767


[Volta ao Índice](#indice)

<div id="ex7"></div>

### Exercício 7:

Existem diversos pontos de relevância para explorar a partir do classificador desenvolvido anteriormente. <br>
Alguns deles englobam:

 * Seria possível empregar outras *features* do passageiro, como `Pclass`, `Age`, entre outras, para influenciar a classificação proposta? Além disso, há a possibilidade de combinar múltiplas *features* para fortalecer a classificação resultante?
 * Foi feita uma avaliação da qualidade do **Classificador Naive-Bayes** na [Análise de Sentimento](#sentimento) utilizando o mesmo conjunto de dados que serviu como base para treinar o classificador. Contudo, é possível afirmar que um novo passageiro será categorizado com a mesma precisão que foi observada na [Análise de Sentimento](#sentimento) apresentada anteriormente?

ESCREVA SUA RESPOSTA AQUI

___

___
<div id="casa"></div>

## Para casa: Estudo Complementar

Ainda, faça as tarefas a seguir:


<div id="classPclass"></div>

### Classificador Naive-Bayes com a variável `Pclass`

Refaça todas as etapas acima feita com a variável `Sex`, mas agora considerando a variável `Pclass`. <br>
Essa outra característica do passageiro classifica melhor se passageiro será ou não sobrevivente?


In [632]:
dados.Pclass.value_counts(True, sort=False)

1st    0.242424
2nd    0.206510
3rd    0.551066
Name: Pclass, dtype: float64

In [633]:
# COLOQUE SEU CODIGO AQUI
prob_class = dados.Pclass.value_counts(True, sort=False)
P_1 = prob_class[0]
P_2 = prob_class[1]
P_3 = prob_class[2]

print(f'Probabilidade de ser da primeira classe: {P_1:0.5g}')
print(f'Probabilidade de ser da segunda classe: {P_2:0.5g}')
print(f'Probabilidade de ser da segunda classe: {P_3:0.5g}')

Probabilidade de ser da primeira classe: 0.24242
Probabilidade de ser da segunda classe: 0.20651
Probabilidade de ser da segunda classe: 0.55107


In [634]:
P_XXXX_dado_S = dados_S.Pclass.value_counts(True, sort=False)
P_1_dado_S = P_XXXX_dado_S[0]
P_2_dado_S = P_XXXX_dado_S[1]
P_3_dado_S = P_XXXX_dado_S[2]

P_XXXX_dado_S

1st    0.397661
2nd    0.254386
3rd    0.347953
Name: Pclass, dtype: float64

In [635]:
P_XXXX_dado_Sc = dados_Sc.Pclass.value_counts(True, sort=False)

P_1_dado_Sc = P_XXXX_dado_Sc[0]
P_2_dado_Sc = P_XXXX_dado_Sc[1]
P_3_dado_Sc = P_XXXX_dado_Sc[2]

In [636]:
P_S_dado_1 = P_1_dado_S*P_S/P_1
P_Sc_dado_1 = 1 - P_S_dado_1
P_S_dado_2 = P_2_dado_S*P_S/P_2
P_Sc_dado_2 = 1 - P_S_dado_2
P_S_dado_3 = P_3_dado_S*P_S/P_3
P_Sc_dado_3 = 1 - P_S_dado_3

print(P_S_dado_1)
print(P_Sc_dado_1)
print(P_S_dado_2)
print(P_Sc_dado_2)
print(P_S_dado_3)
print(P_Sc_dado_3)

0.6296296296296295
0.37037037037037046
0.47282608695652173
0.5271739130434783
0.24236252545824846
0.7576374745417516


In [637]:
map_dict_Pclass = {}
if P_S_dado_1 > P_Sc_dado_1:
    map_dict_Pclass['1st'] = 'Yes'
else:
    map_dict_Pclass['1st'] = 'No'

if P_S_dado_2 > P_Sc_dado_2:
    map_dict_Pclass['2nd'] = 'Yes'
else:
    map_dict_Pclass['2nd'] = 'No'

if P_S_dado_3 > P_Sc_dado_3:
    map_dict_Pclass['3rd'] = 'Yes'
else:
    map_dict_Pclass['3rd'] = 'No'

dados['BootPclass'] = dados['Pclass'].map(map_dict_Pclass)

dados

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,BootSex,BootPclass
0,1,No,3rd,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,No,No
1,2,Yes,1st,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Yes,Yes
2,3,Yes,3rd,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Yes,No
3,4,Yes,1st,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Yes,Yes
4,5,No,3rd,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,No,2nd,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,No,No
887,888,Yes,1st,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,Yes,Yes
888,889,No,3rd,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S,Yes,No
889,890,Yes,1st,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,No,Yes


In [638]:
pd.crosstab(dados.BootPclass, dados.Survived)

Survived,No,Yes
BootPclass,,
No,469,206
Yes,80,136


In [639]:
print((469+136)/(469+206+80+136))
print((80)/(469+80))
print((206)/(206+136))

0.6790123456790124
0.14571948998178508
0.6023391812865497


<div id="testtrain"></div>

### Separando conjunto de dados em Treinamento (train)  e Teste (test)

Para aplicar a teoria de Machine Learning (ML), é necessário separar a base de dados em duas partes: treinamento e teste.

 * **Treinamento**: usualmente contém 75% das linhas (aqui, considere 670 passageiros) para treinar o modelo. Ou seja, qual perfil do passageiro tende a ser sobrevivente e qual perfil, não sobreviveu.
 * **Teste**: as demais linhas (aqui, 221 passageiros). Esses passageiros deverão ser classificacos como sobrevivente ou não sobrevivente segundo modelo treinado. Essa classificação via modelo deve ser confrontada com real classificação fazendo uma Análise de Sentimento, como é chamado pela área de ML.
 
Faça a separação de 670 passageiros para ser da base treinamento e os demais, como teste. Faça essa separação de forma aleatória.

O comando `shuffle` da biblioteca `sklearn.utils` pode ser uma alternativa, por ora, para realizar essa separação (para estudar depois, consulte este [link](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)).


In [640]:
from sklearn.utils import shuffle

In [641]:
dados2 = shuffle(dados).copy()

In [642]:
dados2.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,BootSex,BootPclass
454,455,No,3rd,"Peduzzi, Mr. Joseph",male,NaN,0,0,A/5 2817,8.0500,NaN,S,No,No
169,170,No,3rd,"Ling, Mr. Lee",male,28.0,0,0,1601,56.4958,NaN,S,No,No
373,374,No,1st,"Ringhini, Mr. Sante",male,22.0,0,0,PC 17760,135.6333,NaN,C,No,Yes


In [643]:
train = dados2.iloc[0:670,]
train.shape

(670, 14)

In [644]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,BootSex,BootPclass
454,455,No,3rd,"Peduzzi, Mr. Joseph",male,NaN,0,0,A/5 2817,8.0500,NaN,S,No,No
169,170,No,3rd,"Ling, Mr. Lee",male,28.0,0,0,1601,56.4958,NaN,S,No,No
373,374,No,1st,"Ringhini, Mr. Sante",male,22.0,0,0,PC 17760,135.6333,NaN,C,No,Yes
340,341,Yes,2nd,"Navratil, Master. Edmond Roger",male,2.0,1,1,230080,26.0000,F2,S,No,No
705,706,No,2nd,"Morley, Mr. Henry Samuel (""Mr Henry Marshall"")",male,39.0,0,0,250655,26.0000,NaN,S,No,No


In [645]:
test = dados2.iloc[670:,]
test.shape

(221, 14)

In [646]:
test.head(5)

PassengerId Survived Pclass  \
293          294       No    3rd   
859          860       No    3rd   
885          886       No    3rd   
431          432      Yes    3rd   
857          858      Yes    1st   

                                                  Name     Sex   Age  SibSp  \
293                                Haas, Miss. Aloisia  female  24.0      0   
859                                   Razi, Mr. Raihed    male   NaN      0   
885               Rice, Mrs. William (Margaret Norton)  female  39.0      0   
431  Thorneycroft, Mrs. Percival (Florence Kate White)  female   NaN      1   
857                             Daly, Mr. Peter Denis     male  51.0      0   

     Parch  Ticket     Fare Cabin Embarked BootSex BootPclass  
293      0  349236   8.8500   NaN        S     Yes         No  
859      0    2629   7.2292   NaN        C      No         No  
885      5  382652  29.1250   NaN        Q     Yes         No  
431      0  376564  16.1000   NaN        S     Yes         No  
857      0  113055  26.5500   E17        S      No        Yes

A partir daqui, treine seu classificador com o *DataFrame* `train` e avalie a qualidade do seu classificador com o *DataFrame* `test`.

In [647]:
# COLOQUE SEU CODIGO AQUI

In [648]:
vars = list(dados)

['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'BootSex', 'BootPclass']


[Volta ao Índice](#indice)